In [2]:
from os import path
from collections import Counter


from transformers import BertTokenizer, LongformerTokenizerFast

In [3]:
gap_file = "/home/shtoshni/Research/litbank_coref/data/gap/orig/gap-test.tsv"

In [4]:
tokenizer = LongformerTokenizerFast.from_pretrained('allenai/longformer-large-4096', add_prefix_space=True)


In [5]:
data = open(gap_file).readlines()[1:]
print(len(data))

2000


In [22]:
instances = []
for line in data:
    doc_key, text, pronoun, pronoun_offset, span1, span1_offset, coref1, span2, span2_offset, coref2 = line.strip().split('\t')[:10]
    
    pronoun_offset, span1_offset, span2_offset = int(pronoun_offset), int(span1_offset), int(span2_offset)
    
    pronoun_boundary = (pronoun_offset, pronoun_offset + len(pronoun))
    span1_boundary = (span1_offset, span1_offset + len(span1))
    span2_boundary = (span2_offset, span2_offset + len(span2))
    
    
    print(line)
    
    for span_boundary, coref_label in zip([span1_boundary, span2_boundary], [coref1, coref2]):
        boundaries = sorted([pronoun_boundary, span_boundary], key=lambda x: x[0])
        
        
        first_span = text[0: boundaries[0][0]].strip()
        second_span = text[boundaries[0][0]: boundaries[0][1]].strip()
        third_span = text[boundaries[0][1]: boundaries[1][0]].strip()
        fourth_span = text[boundaries[1][0]: boundaries[1][1]].strip()
        fifth_span = text[boundaries[1][1]:].strip()
        
        
        doc = []
        prefix_len = []
        spans = []
        for idx, intermediate_span in enumerate([first_span, second_span, third_span, fourth_span, fifth_span]):
            prefix_len.append(len(doc))
            span_tokens = tokenizer.tokenize(intermediate_span)
            if idx == 1 or idx == 3:
                spans.append([prefix_len[-1], prefix_len[-1] + len(span_tokens) - 1])
                
#             doc.extend(span_tokens)
            doc.extend(tokenizer.convert_tokens_to_ids(span_tokens))
#             doc.extend(tokenizer.tokenize(intermediate_span))

        
        clusters = []
        if coref_label == 'TRUE':
            clusters = [[spans[0]], [spans[1]]]
        else:
            clusters = [[spans[0], spans[1]]]
            
        print(clusters)
#         for span_start, span_end in spans:
#             print(tokenizer.convert_tokens_to_string(doc[span_start: span_end + 1]))
        
#         print(tokenizer.convert_tokens_to_string(doc))
        
    
    
    break
    
    
    

test-1	Upon their acceptance into the Kontinental Hockey League, Dehner left Finland to sign a contract in Germany with EHC M*nchen of the DEL on June 18, 2014. After capturing the German championship with the M*nchen team in 2016, he left the club and was picked up by fellow DEL side EHC Wolfsburg in July 2016. Former NHLer Gary Suter and Olympic-medalist Bob Suter are Dehner's uncles. His cousin is Minnesota Wild's alternate captain Ryan Suter.	His	383	Bob Suter	352	FALSE	Dehner	366	TRUE	http://en.wikipedia.org/wiki/Jeremy_Dehner

[[[84, 86], [94, 94]]]
[[[88, 89]], [[95, 95]]]
